In [133]:
import cv2
from sklearn.cluster import KMeans
from keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
import numpy as np
import pandas as pd
import os
from math import floor,sqrt
from moviepy.editor import VideoFileClip
import librosa
from sklearn.preprocessing import StandardScaler, LabelEncoder
import h5py
from sklearn.metrics import precision_score, recall_score, f1_score

## Video Summarization

### Annotations loader

In [134]:
# Load annotations
def load_annotations(anno_file, info_file):
    # Read the annotation and info files
    annotations = pd.read_csv(anno_file, sep='\t', header=None)
    info = pd.read_csv(info_file, sep='\t', header=None)
    
    # Rename columns for better understanding
    annotations.columns = ['video_id', 'category', 'importance_score']
    info.columns = ['category_code', 'video_id', 'title', 'url', 'length']
    
    return annotations, info

### Preprocessing and Feature Extraction


#### a. Video Processing and Frame Extraction


In [135]:
def extract_frames(video_path, frame_rate=25):
    video = cv2.VideoCapture(video_path)
    count = 0
    success = True
    frames = []
    
    while success:
        success, image = video.read()
        if count % frame_rate == 0 and success:
            frames.append(image)
        count += 1

    video.release()
    return frames

#### b. Audio Extraction


In [136]:

def extract_audio_from_video(video_path, output_audio_path):
    video = VideoFileClip(video_path)
    audio = video.audio
    audio.write_audiofile(output_audio_path)
    video.close()

#### c.Audio Features

In [137]:
def extract_audio_features(audio_path):
    """
    Extract audio features from an audio file, specifically MFCCs.

    :param audio_path: Path to the audio file.
    :return: Array of MFCCs.
    """
    # Load the audio file
    y, sr = librosa.load(audio_path)

    # Extract MFCCs from the audio
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)  # You can change n_mfcc based on your needs

    # To capture the variation over time, you might compute statistics across MFCCs over time
    # Here, we compute the mean of the MFCCs across time
    mfccs_processed = np.mean(mfccs.T,axis=0)

    return mfccs_processed

#### d. Feature Extraction (example with visual features using a CNN)


In [138]:
model = VGG16(weights='imagenet', include_top=False)

def extract_visual_features(frames):
    features = []
    for frame in frames:
        if frame is not None:
            img = cv2.resize(frame, (224, 224))  # Resize frame to 224x224
            img = img_to_array(img)        # Convert to array
            img = np.expand_dims(img, axis=0)    # Add batch dimension
            img = preprocess_input(img)          # Preprocess for VGG16
            
            feature = model.predict(img,use_multiprocessing=True,workers=4)
            features.append(feature.flatten())

    return features

### Connect Audio/Annotation/Video

#### Padding Features to fit the model

In [139]:
def padding(visual_features, audio_features, annotation_features):
    # inputs are at least 1D arrays
    visual_features = [np.array(v, ndmin=1, dtype=float) for v in visual_features]
    audio_features = np.array(audio_features, ndmin=1, dtype=float)
    annotation_features = [np.array(a, ndmin=1, dtype=float) for a in annotation_features]

    # Find the maximum length of the features
    max_length = max(
        max(v.size for v in visual_features),
        audio_features.size,
        max(a.size for a in annotation_features)
    )

    # Pad features to match the maximum length
    visual_padded = [np.pad(v, (0, max_length - v.size), 'constant') for v in visual_features]
    audio_padded = np.pad(audio_features, (0, max_length - audio_features.size), 'constant')
    annotation_padded = [np.pad(a, (0, max_length - a.size), 'constant') for a in annotation_features]

    return visual_padded, audio_padded, annotation_padded


#### Annotation To List

In [140]:
def annotation2List(annotation_features):
    # Make the string '1,1,1,3,2,2,4,4,1' to float list
    annotation_float_array=[]
    for annotation in annotation_features:
        
        if isinstance(annotation,str):
            annotation = annotation.split(',')
        if isinstance(annotation,list):
            for anno in annotation:
                annotation_float_array.append(float(anno))
        else:
            annotation_float_array.append(float(annotation))
    return annotation_float_array

#### Add weights to ignore padding

In [141]:
def apply_feature_weighting(combined_features, original_feature_length, original_feature_weight=1, padding_weight=0.1):
    """
    Apply different weights to the original and padded parts of the feature vectors.

    :param combined_features: List of combined feature vectors (including padding).
    :param original_feature_length: The length of the original (non-padded) part of the feature vectors.
    :param original_feature_weight: Weight to be applied to the original features.
    :param padding_weight: Weight to be applied to the padded features.
    :return: List of weighted feature vectors.
    """
    weighted_combined_features = []

    for feature in combined_features:
        # Create a weight vector: higher weight for original features, lower for padding
        weights = [original_feature_weight] * original_feature_length
        weights += [padding_weight] * (len(feature) - original_feature_length)
        
        # Apply weights to the feature vector
        weighted_feature = np.array(feature) * np.array(weights)
        weighted_combined_features.append(weighted_feature)

    return weighted_combined_features


#### Extract Data

In [142]:
def extractData(video_path, anno_file, info_file):
    # Extract frames from the video
    frames = extract_frames(video_path) 

    # # Extract visual features
    visual_features = extract_visual_features(frames) 

    # Extract audio
    audio_output_path = 'datasets/extractedAudio/extracted_audio.wav'
    extract_audio_from_video(video_path, audio_output_path) 

    # Extract audio features
    audio_features = extract_audio_features(audio_output_path) 

    # Load annotations
    annotations, info = load_annotations(anno_file, info_file)
    
    # return visual_features, frames
    return annotations,audio_features,visual_features,frames


#### Remove all zeros

In [143]:
def compact_non_zeros(array):
    """
    Move all non-zero elements of an array to the front, keeping their order.

    :param array: Input array.
    :return: Compacted array with non-zeros at the front.
    """
    non_zeros = array[array != 0]
    if len(non_zeros) == 0:  # If the array is all zeros
        print("All zeros")
        return array
    return np.pad(non_zeros, (0, len(array) - len(non_zeros)), 'constant')

#### Kmeans and feature connection

In [144]:
def integrate_and_cluster_features(video_path, anno_file, info_file, num_clusters=None):
    """
    Integrate visual, audio, and annotation features from a video,
    and perform clustering on the combined features.

    :param video_path: Path to the video file.
    :param anno_file: Path to the annotation file.
    :param info_file: Path to the info file.
    :param num_clusters: Number of clusters to use in KMeans.
    :return: Cluster labels for each data point.
    """
    
    # Extract data from video
    # visual_features,frames=extractData(video_path, anno_file, info_file)

    annotations,audio_features,visual_features,frames=extractData(video_path, anno_file, info_file)
    
    # If annotations include categorical data, convert it to numerical format
    # May exclude it later 
    label_encoder = LabelEncoder()
    categorical_columns = ['category']
    for column in categorical_columns:
        if column in annotations.columns:
            annotations[column] = label_encoder.fit_transform(annotations[column])

    # Exclude non-numerical data from annotations if necessary
    annotation_features = annotations.drop(columns=['video_id']).values

    
    # """ 

    # Determine the original feature length
    original_feature_length = len(visual_features) + len(audio_features) + len(annotation_features)

    # Combine features with padding
    combined_features = []
    
    # Remove temporaraily the audio feature
    for i, frame in enumerate(frames):
        
        annotation_float_array = annotation2List(annotation_features[i])
        
        visual_padded, audio_padded, annotation_padded = padding(visual_features, audio_features, annotation_float_array)

        combined_feature = np.concatenate([
            np.array(visual_padded[i], dtype=float),
            np.array(audio_padded, dtype=float),
            np.array(annotation_padded[i], dtype=float),
        ])
        combined_features.append(combined_feature)

    # Apply feature weighting
    weighted_combined_features = apply_feature_weighting(combined_features, original_feature_length)

    # Convert to 2D NumPy array and normalize
    combined_features_array = np.array(weighted_combined_features)
    
    combined_features_normalized = StandardScaler().fit_transform(combined_features_array)
    # """
    

    # Calculate the number of clusters if not provided
    if num_clusters is None:
        num_clusters = floor(sqrt(len(combined_features_normalized)))
    
    
    # Perform clustering
    print("Performing clustering...")
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(combined_features_normalized)
    
    return [frames, kmeans.labels_]
    # return the frames and the labels


### Summarization

#### Find bigger cluster

In [145]:
from collections import Counter
def getBiggestCluster(labels):
    cluster_counts = Counter(labels)

    # Find the cluster with the maximum number of frames
    return max(cluster_counts, key=cluster_counts.get),cluster_counts

#### Frames Selection

In [146]:
def frame_selection(frames, labels):
    """
    Create a summary by selecting frames from the most populous cluster.

    :param frames: List of frames.
    :param labels: Cluster labels for each frame.
    :return: List of frames belonging to the most populous cluster.
    """
    # Count the number of frames in each cluster
    max_cluster,cluster_counts = getBiggestCluster(labels)

    # Select frames belonging to the most populous cluster
    summary_frames = [frame for frame, label in zip(frames, labels) if label == max_cluster]

    print(f"Number of frames selected for summary: {len(summary_frames)}, Cluster: {max_cluster}")
    # print all clusters len
    print(f"Cluster counts: {cluster_counts}")
    return summary_frames


#### Video Creator

In [147]:
def create_video_from_frames(frames, output_path, frame_rate=25):
    if not frames:
        print("No frames to create a video.")
        return 
    # Determine the width and height from the first frame
    height, width, layers = frames[0].shape

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, frame_rate, (width, height))

    # Write each frame to the video
    for frame in frames:
        out.write(frame)

    # Release the VideoWriter object
    out.release()

#### Load .Mat file

In [148]:
def decode_titles(encoded_titles, hdf5_file):
    decoded_titles = []
    for ref_array in encoded_titles:
        # Handle the case where each ref_array might contain multiple references
        for ref in ref_array:
            # Dereference each HDF5 object reference to get the actual data
            title_data = hdf5_file[ref]
            # Decode the title
            decoded_title = ''.join(chr(char[0]) for char in title_data)
            decoded_titles.append(decoded_title)
    return decoded_titles


def load_mat_file(file_path,videoID):
    """
    Load a .mat file and return its contents.

    :param file_path: Path to the .mat file.
    :return: Contents of the .mat file.
    """
    with h5py.File(file_path, 'r') as file:
        user_anno_refs=file['tvsum50']['user_anno'][:]
        video_refs=file['tvsum50']['video'][:]

        decoded_videos = decode_titles(video_refs,file)
    
        annotations = []
        
        # Get the index from decoded video list to find the annotation for the video
        index = [i for i, x in enumerate(decoded_videos) if x.lower() in videoID.lower()][0]
        
        # Iterate over each reference
        for ref in user_anno_refs:
            # Dereference each HDF5 object reference
            ref_data = file[ref[0]]

            # Convert to NumPy array and add to the annotations list
            annotations.append(np.array(ref_data))
            
        return annotations[index]

#### f1score with ground_truth

In [149]:
def kmeans_f1score(ground_truth_path, summary_frames, videoID):
    # Load ground truth summary
    ground_truth = np.array(load_mat_file(ground_truth_path, videoID))
    
    summary_frames=np.array(summary_frames)
    
    # Average scores across annotators, if necessary
    avg_ground_truth_scores = np.mean(ground_truth, axis=0) if ground_truth.ndim > 1 else ground_truth

    # threshold
    threshold = 1  # Ensure this threshold is suitable for your dataset
    
    # Convert ground truth to binary labels
    binary_ground_truth = np.where(avg_ground_truth_scores >= threshold, 1, 0)

    # Convert selected frames to binary labels
    selected_frames_binary = np.zeros_like(binary_ground_truth)
    
    # Assuming summary_frames is a list of indices of frames selected for the summary
    for index in summary_frames:
        for frame_index in summary_frames:
            selected_frames_binary[frame_index] = 1

    # Calculate F1-Score
    precision = precision_score(binary_ground_truth, selected_frames_binary)
    recall = recall_score(binary_ground_truth, selected_frames_binary)
    f1 = f1_score(binary_ground_truth, selected_frames_binary)
    
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1-Score:", f1)
    return f1


# Code to create summary video

#### Variables

In [150]:
annotation_path='datasets/ydata-tvsum50-v1_1/data/ydata-tvsum50-anno.tsv'
info_path='datasets/ydata-tvsum50-v1_1/data/ydata-tvsum50-info.tsv'

In [151]:
video_path='datasets/ydata-tvsum50-v1_1/video/'
summary_video_path='datasets/summary_videos/'

In [152]:
ground_truth_path='datasets/ydata-tvsum50-v1_1/ground_truth/ydata-tvsum50.mat'

##### Get the list of the videos in the folder

In [153]:
video_list = [video for video in os.listdir(video_path) if video.endswith('.mp4')]  # List comprehension

#### Function for videoSummarizion

In [154]:
def videoSummarize(annotation_path=None, info_path=None, video_path=None, summary_video_path=None,video_list=None):
    for video in video_list:  
        frames,labels = integrate_and_cluster_features(video_path+video, annotation_path, info_path)
        
        # Select Representative Frames
        summary_frames = frame_selection(frames, labels)  # Select frames from the most populous cluster
        
        # Create Summary Video
        create_video_from_frames(summary_frames, summary_video_path+video,30) # Create a video from the selected frames
        
        kmeans_f1score(ground_truth_path,summary_frames,video.split('.')[0])
        break
        

#### Code for dynamic annotation and video summarization

In [155]:
videoSummarize(annotation_path, info_path, video_path, summary_video_path, video_list)

1/1 [==============================] - 0s 169ms/step
MoviePy - Writing audio in datasets/extractedAudio/extracted_audio.wav


MoviePy - Done.
Performing clustering...


TypeError: Singleton array 3 cannot be considered a valid collection.

#### Results
##### only using visual features 

for index in summary_frames: 
    selected_frames_binary[index] = 1
Precision: 0.06076388888888889
Recall: 1.0
F1-Score: 0.11456628477905074

##### only audio+visual+annotation
Precision: 0.06076388888888889
Recall: 1.0
F1-Score: 0.11456628477905074

using PCA from 4 dim to 2dim in visual features
Precision: 1.0
Recall: 0.0013020833333333333
F1-Score: 0.002600780234070221


for index in summary_frames: 
        for frame_index in index:
            selected_frames_binary[frame_index] = 1
Precision: 1.0
Recall: 0.037037037037037035
F1-Score: 0.07142857142857142